In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("spark://192.168.2.97:7077") \
        .appName("Group-6-Question-1-Claude")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.cores.max", 2)\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/12 15:29:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/12 15:29:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.functions import trim, regexp_replace, lower, explode, trim, split
from pyspark.sql.types import StructType, StructField, StringType

In [3]:
def parse_json_body_to_string(json_filename):
    # Read the json file
    df = spark.read.json(json_filename)

    # Make everything lowercase
    my_df_lower = df.select(lower("body").alias("lowercase_body"))

    # Remove all charachters that exist in charachters_to_delete
    charachters_to_delete = "[\.,\[\]\(\):_\-!?\'\+=;/&{}@$#*\"\\\\%><|~¨´¤]"

    my_df_clean = my_df_lower.withColumn("lowercase_body", regexp_replace(my_df_lower.lowercase_body, charachters_to_delete, ""))

    # Parse the body to words
    my_df_words = my_df_clean.select(explode(split("lowercase_body", "\s+")).alias("word"))

    return my_df_words

In [5]:
word_count = {}
years = ["06", "07"]
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

df_all_years = parse_json_body_to_string("./RC_empty.json")

for year in years:
    df_this_year = parse_json_body_to_string("./RC_empty.json")

    # Append all the months to the dataframe
    for month in months:
        df_month = parse_json_body_to_string("../RC_20" + year + "-" + month + ".json")
        df_this_year = df_this_year.union(df_month)
    
    # Count and sort the words. Save the result in word_count
    word_count_this_year = df_this_year.groupBy("word").count().orderBy("count", ascending=False)
    # word_count_this_year = word_count_this_year.sort('count', reverse=False)
    year_string = "20" + year
    word_count.update({year_string: word_count_this_year})

    # Update df_all_years
    df_all_years = df_all_years.union(df_this_year)

# Find top 1000 most used words
word_count_all_years = df_all_years.groupBy("word").count().orderBy("count", ascending=False)
# word_count_all_years = word_count_all_years.sort('count', reverse=False)
word_count.update({"all_years": word_count_all_years})
most_common_words = word_count["all_years"].limit(10)

# Remove the most common words from the word count for each year
word_count_unique = {}
for year in years:
    year_string = "20" + year
    word_count_unique.update({year_string: word_count[year_string].subtract(most_common_words)})
    # word_count_unique[year_string] = word_count_unique[year_string].sort('count', reverse=True)

23/03/10 19:47:33 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (192.168.2.154 executor 0): java.io.FileNotFoundException: 
File file:/home/ubuntu/Data-Engineering-Group-6/RC_empty.json does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:104)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Ite

Py4JJavaError: An error occurred while calling o52.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (192.168.2.154 executor 0): java.io.FileNotFoundException: 
File file:/home/ubuntu/Data-Engineering-Group-6/RC_empty.json does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at org.apache.spark.sql.execution.datasource

In [ ]:
# word_count["2006"].show(10)
# word_count["all_years"].show(10)
word_count_unique["2006"].show(10)
word_count_unique["2007"].show(10)

In [ ]:
spark_session.stop()